# MIGHTEE-HI Data: Calculating $W_{50}$
***
By Wanga Mulaudzi 

<b>Date:</b> 1 August 2021
<br>
<b>Affiliation:</b> Department of Astronomy, University of Cape Town, Private Bag X3, Rondebosch 7701, South Africa
<br>
<b>Contact:</b> MLDWAN001@myuct.ac.za
<br>
<b>Ilifu Jupyter Kernel:</b> PyMultiNest

This notebook reads in text files created by the `Step2_Masked_Profiles_COSMOS_1330.ipynb` notebook and then fits a Busy Function using ```PyMultiNest``` (see http://johannesbuchner.github.io/pymultinest-tutorial/index.html).

To find the width at $20\%$ and $50\%$, we have to decipher if the profile is double horned or not, and based on that, we can find the widths. Since the data from this particular cube is very low resolution, the best width to calculate is $W_{50}$, else we would be reaching into the noise of the profile.

The busy function is defined as

\begin{align}
B(x) = \frac{a}{4}[\text{erf}(b_1\{w+x-x_e\})+1]\times[\text{erf}(b_2\{w-x+x_e\})+1]\times [c|x-x_p|^n+1], \\
\end{align}

where $\text{erf}(x) = \frac{2}{\sqrt{\pi}}\int^x_0\text{exp}(-t^2)\text{d}t$ is the gaussian error function, $a$ is the scaling factor, $b_1$ and $b_2$ the steepness of the line flanks, $w$ the half-width of the profile, $x_e$ and $x_p$ the centroid of the error functions/polynomial, $c$ the scaling factor of the polynomial trough, and $n$ the order of the polynomial (see https://www.astron.nl/phiscc2014/Documents/Techniques/Westmeier_Busy_Function_phiscc2014.pdf for more details). The diagram below further illustrates the different parameters.

![BF.png](BF.png)

In [1]:
# Import statements
from astropy.io import ascii
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import matplotlib.ticker as mtick
import numpy as np
from numpy import log, exp, pi
import os
import pymultinest
import pylab as pl
import scipy as sp
import scipy.stats
import shutil

In [2]:
# # Initiate parameters for plotting
# pl.rc('axes',titlesize='large')
# pl.rc('text', usetex=False)
# pl.rc('font', **{'family':'serif','size':20})
# pl.rc('axes', labelsize=16)
# pl.rc('xtick',labelsize=16)
# pl.rc('ytick',labelsize=16)

# plt.rcParams['mathtext.fontset'] = 'custom'
# plt.rcParams['mathtext.rm'] = 'serif'
# plt.rcParams['font.family'] = 'serif'

## 1. User Inputs and Creating a Directory

In [3]:
# Path to your detection file list in an ASCII table format (e.g xxx.dat)
detections_list = 'COSMOS123_1330_catalogue_notes.txt'
detections = ascii.read(detections_list)

# Path for new 'analysis_output' directory that will be created
dirName = 'analysis_output/' 

## 2. Busy Function

In [5]:
# Define the busy function model
def Bmodel(a, b1, b2, w, xe, xp, c, n):
    '''
    x is the list of x-values
    a is the scaling factor
    b1 is the steepness of the first flank
    b2 is the steepness of the second flank
    w is half the FWHM
    xe is the center of the error function
    xp is the center of the polynomial
    c is the scaling factor of the polynomial trough
    n is the order of the polynomial
    '''
    return (a/4.)*(sp.special.erf(b1*(w+vel-xe))+1)*(sp.special.erf(b2*(w-vel+xe))+1)*(c*pow(abs(vel-xp),n)+1)

In [6]:
# Define the busy function model for the model solutions that take in new x
def Bmodelx(x, a, b1, b2, w, xe, xp, c, n):
    '''
    x is the list of x-values
    a is the scaling factor
    b1 is the steepness of the first flank
    b2 is the steepness of the second flank
    w is half the FWHM
    xe is the center of the error function
    xp is the center of the polynomial
    c is the scaling factor of the polynomial trough
    n is the order of the polynomial
    '''
    return (a/4.)*(sp.special.erf(b1*(w+x-xe))+1)*(sp.special.erf(b2*(w-x+xe))+1)*(c*pow(abs(x-xp),n)+1)

## 3. The prior and the log likelihood functions

In [7]:
# Define the prior that transforms the unit cube into the parameter cube that is now in log space
# parameters are a, b1, b2, w, xe, xp, c, n
# uniform priors span one order of magnitude
# log-uniform priors span multiple orders of magnitude. Should be written in powers of 10
def prior(cube, ndim, nparams):
    cube[0] = cube[0] # uniform prior for a between 0*1=0 and 1*1=1
    cube[1] = cube[1] # uniform prior for b1 between 0*1=0 and 1*1=1
    cube[2] = cube[2] # uniform prior for b2 between 0*1=0 and 1*1=1
    cube[3] = 10**(cube[3]*3) # log-uniform prior for w between 10**(0*3)=1 and 10**(1*3)=1e3
    cube[4] = -cube[4]*600 + 300 # uniform prior for xe between -0*600+300=300 and -1*600+300=-300
    cube[5] = -cube[5]*600 + 300 # uniform prior for xp between -0*600+300=300 and -1*600+300=-300
    cube[6] = 10**(-cube[6]*2 - 7) # log-uniform prior for c between 10**(-0*2-7)=1e-7 and 10**(-1*2-7)=1e-9
    cube[7] = 6*cube[7] + 2 # log-uniform prior for n between 6(0)+2=2 and 6(1)+2=8

In [8]:
# Define the loglike function which returns the logarithm of the likelihood (least squares)
def loglike(cube, ndim, nparams):
    # Assign each of the parameters
    a, b1, b2, w, xe, xp, c, n = cube[0], cube[1], cube[2], cube[3], cube[4], cube[5], cube[6], cube[7]
    
    # Evaluate the velocity data with the parameters
    fluxmodel = Bmodel(a, b1, b2, w, xe, xp, c, n)
    
    flux2 = np.array(phi) # converting flux from a list to an array
    noise2 = np.array(noise)
    
    # Calculate the sum of squares normalised by the residuals
    chi2 = (((fluxmodel_model - fluxdata) / noise2)**2).sum()

    # The log likelihood function
    loglikelihood = -((np.log(2*np.pi*noise2**2)).sum())/2 - chi2/2 # since noise is not constant, the sum over sigma is not just N=len(flux2)

    return loglikelihood

## 4. Running PyMultiNest

## First Run

In [9]:
# Need to create a list to store all the data itself
# for plotting the model solutions later on
all_flux = []
all_flux_err = []
all_max_flux = []
all_vel = []
all_cvel = []
all_sdv = []
all_sdv_err = []

In [ ]:
# Path to Step2_Masked_profiles_COSMOS_1330.ipynb file output
path_to_data = ''

# Loop through each detection
for i in range(len(detections)):
    # Create a directory for detection i's results
    directory_i = dirName+'detection_'+str(detections[i]['Detection'])+'/'
    
    if not os.path.exists(directory_i):
        os.makedirs(directory_i)
    else:
        shutil.rmtree(directory_i)
        os.makedirs(directory_i)
        
    # Read in the data
    txt_file = path_to_data+'COSMOS_1330_masked_detection'+str(detections[i]['Detection'])+'_profile_vel.txt'
    
    data = np.genfromtxt(txt_file, delimiter=' ').T
    cvel = data[1][0]
    vel = data[0][2:] - cvel
    fluxdata = data[1][2:]/max(data[1][2:])
    fluxerr = data[2][2:]/max(data[1][2:])
    intflux = data[1][1]
    intfluxerr = data[2][1]
    
    # Save the data
    all_flux.append(fluxdata)
    all_flux_err.append(fluxerr)
    all_max_flux.append(max(data[1][2:]))
    all_vel.append(vel)
    all_cvel.append(cvel)
    all_sdv.append(intflux)
    all_sdv_err.append(intfluxerr)
    
    noise = fluxerr
    
    # Number of dimensions our problem has
    parameters = [r'$a$', r'$b_1$', r'$b_2$', r'$w$', r'$x_e$', r'$x_p$', r'$c$', r'$n$']
    n_params = len(parameters)
    
    # Run PyMultiNest
    pymultinest.run(loglike, prior, n_params, outputfiles_basename='detection_'+str(detections[i]['Detection']), resume=False, verbose=True)
    json.dump(parameters, open(directory_i+'/params.json', 'w')) # Save the parameters
    
    # plot the distribution of a posteriori possible models
    pl.figure(figsize=(15,7)) 
    pl.step(vel, fluxdata, '-', color='black', where='mid')
    pl.errorbar(vel, fluxdata, yerr=fluxerr, fmt='', marker=None, ls='none', color='k')
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    pl.tight_layout()
    
    # Plot the distribution of a posteriori possible models
    # Modelx is just a linear space of the velocities
    modelx = np.linspace(min(vel), max(vel), 1000)

    an = pymultinest.Analyzer(outputfiles_basename='detection_'+str(detections[i]['Detection']), n_params = n_params)
    
    # Loop through each possible model and plot it
    for (a, b1, b2, w, xe, xp, c, n) in an.get_equal_weighted_posterior()[::100,:-1]:
        pl.plot(modelx, Bmodelx(modelx, a, b1, b2, w, xe, xp, c, n), '-', color='red', alpha=0.3)
        
        # Axes parameters
        pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
        pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)

    pl.savefig(directory_i+'posterior'+str(detections[i]['Detection'])+'.png')
    pl.close()

PyMultiNest dumps all the output where the notebook is located so we need to re-organize everything. May need to run this cell twice if errors arise.

In [12]:
# Directory to where the MultiNest data dump is
source = ''

# Read in all the files in the directory
files = os.listdir(source)

# Detections with two digit numbers confuse python so sort detections in reverse order
rev_det = [detections[i]['Detection'] for i in range(len(detections))][::-1]

# Loop through each detection
for i in rev_det:
    # Path to detection folder
    dest = dirName+'detection_'+str(i) 
    
    # Loop through all the files
    for file in files:
        if (file.startswith('detection_'+str(i))):
            shutil.move(file, dest)

## 5. Marginals and best parameters plot

In [13]:
# List to store median and average parameters for each profile
med_best_par_list = []
av_best_par_list = []

In [14]:
# List to store 3 sigma values for a
three_sigma_a = []

In [ ]:
# Create the marginal plots for each detection
for d in range(len(detections)):
    # Path to detection folder
    dest = dirName+'detection_'+str(detections[d]['Detection'])+'/'
    
    # Load the parameters
    params = json.load(open(dest+'params.json'))
    n_params = len(params)
    
    # Create the analyzer object
    an = pymultinest.Analyzer(n_params = n_params, outputfiles_basename = dest+'detection_'+str(detections[d]['Detection']))
    
    # Get a dictionary containing information about the logZ and its errors,
    # the individual modes and their parameters, quantiles of the parameter posteriors
    stats = an.get_stats()
    
    # get the best fit (highest likelihood) point
    bestfit_params_dict = an.get_best_fit()
    bestfit_params = bestfit_params_dict.get('parameters')
    
    json.dump(stats, open(dest+'stats.json', 'w+'))
    
    # Create the marginal plot
    p = pymultinest.PlotMarginal(an)
    
    values = an.get_equal_weighted_posterior()
    assert n_params == len(stats['marginals'])
    modes = stats['modes']
    
    # Dimensionality of the plot
    dim2 = os.environ.get('D', '1' if n_params > 20 else '2') == '2'
    nbins = 100 if n_params < 3 else 20
    
    # Set up grid layout
    params_count2 = [0, 1, 2, 3, 4, 5, 6] # Grid for j
    params_count_indx = 0
    
    if dim2:
        plt.figure(figsize=(27*n_params, 27*n_params))
        #for i in range(n_params):
        for i in range(1, n_params+1, 1):
            stepsizes = [0.1, 0.35, 0.35, 10, 10, 50, 4e-8, 0.4]
            axis_formatters = ['%0.1f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
            plt.subplot(n_params, n_params, (1 + 9*(i-1)))
            plt.subplots_adjust(hspace=0.4, wspace=0.4)
            
            plt.xlabel(parameters[i-1], fontsize=200)

            m = stats['marginals'][i-1]
            
            # Store 3 sigma range for a
            if i-1 == 0:
                three_sigma_a.append(m['3sigma'])
            
            plt.xlim(m['5sigma'])
            plt.xticks(rotation=40)

            oldax = plt.gca()
            x,w,patches = oldax.hist(values[:,i-1], bins=nbins, edgecolor='grey', color='grey', histtype='stepfilled', alpha=0.2)
            oldax.set_ylim(0, x.max())
            oldax.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
            oldax.tick_params(axis='x', labelbottom=False, labeltop=True)
            
            startax, endax = oldax.get_xlim()
            oldax.xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
            oldax.xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
            oldax.xaxis.set_label_position('top')
            
            plt.setp(oldax.get_xticklabels(), fontsize=160)
            plt.setp(oldax.get_yticklabels(), fontsize=160)
            plt.setp(oldax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(oldax.yaxis.get_majorticklabels(), fontsize=160)

            newax = plt.gcf().add_axes(oldax.get_position(), sharex=oldax, frameon=False)
            newax.tick_params(axis='x', labelbottom=False, labeltop=True)
            plt.setp(newax.get_xticklabels(), fontsize=160)
            plt.setp(newax.get_yticklabels(), fontsize=160)
            p.plot_marginal(i-1, ls='-', color='blue', linewidth=3)
            newax.set_ylim(0, 1)

            ylim = newax.get_ylim()
            y = ylim[0] + 0.05*(ylim[1] - ylim[0])
            center = m['median']
            low1, high1 = m['1sigma']
            #print(center, low1, high1)
            newax.errorbar(x=center, y=y,
                xerr=np.transpose([[center - low1, high1 - center]]), 
                color='blue', linewidth=2, marker='s')
            oldax.set_yticks([])
            #newax.set_yticks([])
            oldax.tick_params(axis='both', length=30, direction='in', right=True)
            newax.tick_params(axis='both', length=30, direction='in', right=True, pad=30)
            newax.yaxis.set_label_position('right')
            newax.xaxis.set_label_position('top')
            newax.yaxis.set_ticks(np.arange(0.2, 1.2, 0.2))
            newax.tick_params(axis='y', labelleft=False, labelright=True)
            newax.set_ylabel("Probability", fontsize=200, labelpad=30)
            plt.setp(newax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(newax.yaxis.get_majorticklabels(), fontsize=160)
            ylim = oldax.get_ylim()
            newax.set_xlim(m['5sigma'])
            oldax.set_xlim(m['5sigma'])
            #plt.close()
            
            if i == n_params:
                break
                
            #for j in range(i):
            for j in range(params_count2[params_count_indx], n_params-1):
                stepsizes = [0.05, 0.4, 0.35, 10, 20, 40, 4e-8, 0.4]
                axis_formatters = ['%0.2f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
                plt.subplot(n_params, n_params, ((n_params + i) + j*n_params))
                p.plot_conditional(i-1, j+1, bins=20, cmap = pl.cm.gray_r)
                for m in modes:
                    plt.errorbar(x=bestfit_params[i-1], y=bestfit_params[j+1], xerr=m['sigma'][i-1], yerr=m['sigma'][j+1])
                    
                    ########
                if j == 6 and i != 1:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j != 6:
                    plt.ylabel(parameters[j+1], fontsize=200)
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j == 6:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    plt.ylabel(parameters[j+1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                else:   
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
    
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                        
                    plt.gca().axes.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                plt.xticks(rotation=40)
                plt.setp(plt.gca().get_xticklabels(), fontsize=160)
                plt.setp(plt.gca().get_yticklabels(), fontsize=160)
                #plt.savefig('cond_%s_%s.pdf' % (params[i], params[j]), bbox_tight=True)
                #plt.close()
                
            params_count_indx += 1
        
        plt.savefig(dest + 'marg'+str(detections[d]['Detection'])+'.pdf')
        #plt.savefig(dest + 'marg'+str(detections[d]['Detection'])+'.png')
        plt.close()
    
    '''
    Storing the average parameter values
    '''
    # Open a text file to save the parameter results
    av_results_txt = open(dest+'av_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')

    # List to store best parameters
    av_best_par = []
    
    h = 0
    for par in parameters:
        av_results_txt.write('%s %.15f %.15f\n' % (par, modes[0]['mean'][h], modes[0]['sigma'][h]))
        av_best_par.append([modes[0]['mean'][h], modes[0]['sigma'][h]])
        h+=1

    av_results_txt.close()
    av_best_par_list.append(av_best_par)
    
    '''
    Choosing the median
    '''
    # Open a text file to save the parameter results_txt
    med_results_txt = open(dest+'med_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    # Write the marginal likelihood
    med_results_txt.write('lnZ %.1f %.1f\n'%(stats['global evidence'], stats['global evidence error']))
    
    # List to store best parameters
    med_best_par = []
    
    # Write the parameters
    for par, marg in zip(parameters, stats['marginals']):
        # 1 sigma provides the lowest and highest estimates
        lo, hi = marg['1sigma']
        sigma = (hi - lo) / 2 # Calculate the error
        
        med = marg['median']
        
        if sigma == 0:
            j = 3
        else:
            j = max(0, int(-np.floor(np.log10(sigma))) + 1)
            
        # Create the string to write    
        fmt = '%%.%df' % j
        fmts = '\t'.join(['%-3s' + fmt + " +- " + fmt +'\n'])
        
        med_results_txt.write('%s %.15f %.15f\n' % (par, med, sigma))
        
        # Save the value and error for this parameter
        med_best_par.append([med, sigma])
    
    med_results_txt.close()
    med_best_par_list.append(med_best_par)
    
    '''
    Model and integrated flux
    '''
    # The busy function with the best parameter
    modelx = np.linspace(min(all_vel[d]), max(all_vel[d]), 1000)
    modely = Bmodelx(modelx, med_best_par[0][0], med_best_par[1][0], med_best_par[2][0], med_best_par[3][0], 
                            med_best_par[4][0], med_best_par[5][0], med_best_par[6][0], med_best_par[7][0])
    
    # Integrated flux of data
    Sdv_data = all_sdv[d]
    Sdv_data_err = all_sdv_err[d]
    
    # Integrated flux of model
    dvmodel = np.average(np.diff(modelx))
    Sdv_model = np.sum(modely*dvmodel)*all_max_flux[d]
    
    '''
    Plotting the median and mean parameters
    '''
    pl.figure(figsize=(15,7)) 
    pl.step(all_vel[d], all_flux[d], 'k', where='mid')
    pl.errorbar(all_vel[d], all_flux[d], yerr=all_flux_err[d], fmt='', marker=None, ls='none', color='k')
    pl.plot(modelx, modely, 'r-')
#     pl.plot(modelx, Bmodelx(modelx, av_best_par[0][0], av_best_par[1][0], av_best_par[2][0], av_best_par[3][0], 
#                             av_best_par[4][0], av_best_par[5][0], av_best_par[6][0], av_best_par[7][0]), 'b-', 
#             alpha=0.5, label=r'Mean W$_{50}$ = %.2f $\pm$ %.2f km/s'%(2*av_best_par[3][0], 2*av_best_par[3][1]))
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Axes parameters
    pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
    pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)
    #pl.legend(loc='upper right', fontsize=17)
    
    # Anchored text
    textstr = '\n'.join((
    r'Med. W$_{50}$ = %.2f $\pm$ %.2f km s$^{-1}$' % (2*med_best_par[3][0], 2*med_best_par[3][1], ),
    r'$\int S\mathrm{dv}_\mathrm{data}$ = %.3f $\pm$ %.3f Jy km s$^{-1}$' % (Sdv_data, Sdv_data_err, ),
    r'$\int S\mathrm{dv}_\mathrm{model}$ = %.3f Jy km s$^{-1}$' % (Sdv_model, )))
    pl.annotate(textstr, xy=(0.03, 0.7), xycoords='axes fraction', fontsize=17)
    
    pl.savefig(dest+'best_par'+str(detections[d]['Detection'])+'.png')
    pl.close()

It is important to look at the marginals to see how each of the paramaters behave as a function of each other. We will have to modify the priors of $a$ for each profile to better constrain the fits.

## Second run with modified priors for $a$ and $x_e$ fixed on median

In [ ]:
path_to_data = '/users/wanga/mightee/analysis/COSMOS_1330/masked_profiles_COSMOS_1330_output/'

# Loop through each detection
for i in range(len(detections)):
    # Define the prior with a different paramater space for a
    # Define the prior that transforms the unit cube into the parameter cube that is now in log space
    # parameters are a, b1, b2, w, xe, xp, c, n
    def prior(cube, ndim, nparams):
        cube[0] = cube[0]*(three_sigma_a[i][1] - three_sigma_a[i][0]) + three_sigma_a[i][0] # modified prior for a within 3 sigma
        cube[1] = cube[1] # log-uniform prior for b1 between 0 and 1
        cube[2] = cube[2] # log-uniform prior for b2 between 0 and 1
        cube[3] = 10**(cube[3]*3) # log-uniform prior for w between 10**(0*3)=1 and 10**(1*3)=1e3
        cube[4] = med_best_par_list[i][4][0] # xe fixed on medians of first run
        cube[5] = -cube[5]*600 + 300 # log-uniform prior for xp between -0*600+300=300 and -1*600+300=-300
        cube[6] = 10**(-cube[6]*2 - 7) # log-uniform prior for c between 10**(-0*2-7)=1e-7 and 10**(-1*2-7)=1e-9
        cube[7] = 6*cube[7] + 2 # log-uniform prior for n between 6(0)+2=2 and 6(1)+2=8
    
    # Create a directory for detection i's results
    directory_i = dirName+'/detection_'+str(detections[i]['Detection'])+'/'
    
    vel = all_vel[i]
    fluxdata = all_flux[i]
    fluxerr = all_flux_err[i]
    
    noise = fluxerr
    
    # Number of dimensions our problem has
    parameters = [r'$a$', r'$b_1$', r'$b_2$', r'$w$', r'$x_e$', r'$x_p$', r'$c$', r'$n$']
    n_params = len(parameters)
    
    # Run PyMultiNest
    pymultinest.run(loglike, prior, n_params, outputfiles_basename='mod_a_med_xe_detection_'+str(detections[i]['Detection']), resume=False, verbose=True)
    json.dump(parameters, open(directory_i+'/mod_a_med_xe_params.json', 'w')) # Save the parameters
    
    # plot the distribution of a posteriori possible models
    pl.figure(figsize=(15,7)) 
    pl.step(vel, fluxdata, '-', color='black', where='mid')
    pl.errorbar(vel, fluxdata, yerr=fluxerr, fmt='', marker=None, ls='none', color='k')
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Plot the distribution of a posteriori possible models
    # Modelx is just a linear space of the velocities
    modelx = np.linspace(min(vel), max(vel), 1000)

    an = pymultinest.Analyzer(outputfiles_basename='mod_a_med_xe_detection_'+str(detections[i]['Detection']), n_params = n_params)
    
    # Loop through each possible model and plot it
    for (a, b1, b2, w, xe, xp, c, n) in an.get_equal_weighted_posterior()[::100,:-1]:
        pl.plot(modelx, Bmodelx(modelx, a, b1, b2, w, xe, xp, c, n), '-', color='red', alpha=0.3)
        
        # Axes parameters
        pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
        pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)

    pl.savefig(directory_i+'mod_a_med_xe_posterior'+str(detections[i]['Detection'])+'.png')
    pl.close()

Moving the output of PyMultiNest into the relevant directories. May need to run this cell twice if errors arise.

In [18]:
source = '/users/wanga/mightee/analysis/COSMOS_1330/'

# Read in all the files in the directory
files = os.listdir(source)

# Detections with two digit numbers confuse python so sort detections in reverse order
rev_det = [detections[i]['Detection'] for i in range(len(detections))][::-1]

# Loop through each detection
for i in rev_det:
    # Path to detection folder
    dest = dirName+'detection_'+str(i) 
    
    # Loop through all the files
    for file in files:
        if (file.startswith('mod_a_med_xe_detection_'+str(i))):
            shutil.move(file, dest)

In [19]:
# List to store median and average parameters for each profile
mod_a_med_xe_med_best_par_list = []
mod_a_med_xe_av_best_par_list = []

In [ ]:
# Create the marginal plots for each detection
for d in range(len(detections)):
    # Path to detection folder
    dest = dirName+'detection_'+str(detections[d]['Detection'])+'/'
    
    # Load the parameters
    params = json.load(open(dest+'mod_a_med_xe_params.json'))
    n_params = len(params)
    
    # Create the analyzer object
    an = pymultinest.Analyzer(n_params = n_params, outputfiles_basename = dest+'mod_a_med_xe_detection_'+str(detections[d]['Detection']))
    
    # Get a dictionary containing information about the logZ and its errors,
    # the individual modes and their parameters, quantiles of the parameter posteriors
    #stats = an.get_stats()
    stats = an.get_stats()
    
    # get the best fit (highest likelihood) point
    bestfit_params_dict = an.get_best_fit()
    bestfit_params = bestfit_params_dict.get('parameters')
    
    # get the best fit (highest likelihood) point
    bestfit_params_dict = an.get_best_fit()
    bestfit_params = bestfit_params_dict.get('parameters')
    
    json.dump(stats, open(dest+'mod_a_med_xe_stats.json', 'w+'))
    
    # Create the marginal plot
    p = pymultinest.PlotMarginal(an)
    
    values = an.get_equal_weighted_posterior()
    assert n_params == len(stats['marginals'])
    modes = stats['modes']
    
    # Dimensionality of the plot
    dim2 = os.environ.get('D', '1' if n_params > 20 else '2') == '2'
    nbins = 100 if n_params < 3 else 20
    
    # Set up grid layout
    params_count2 = [0, 1, 2, 3, 4, 5, 6] # Grid for j
    params_count_indx = 0
    
    if dim2:
        plt.figure(figsize=(27*n_params, 27*n_params))
        #for i in range(n_params):
        for i in range(1, n_params+1, 1):
            stepsizes = [0.1, 0.35, 0.35, 10, 10, 50, 4e-8, 0.4]
            axis_formatters = ['%0.1f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
            plt.subplot(n_params, n_params, (1 + 9*(i-1)))
            plt.subplots_adjust(hspace=0.4, wspace=0.4)
            
            plt.xlabel(parameters[i-1], fontsize=160)

            m = stats['marginals'][i-1]
            
            plt.xlim(m['5sigma'])
            plt.xticks(rotation=40)

            oldax = plt.gca()
            x,w,patches = oldax.hist(values[:,i-1], bins=nbins, edgecolor='grey', color='grey', histtype='stepfilled', alpha=0.2)
            oldax.set_ylim(0, x.max())
            oldax.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
            oldax.tick_params(axis='x', labelbottom=False, labeltop=True)
            
            startax, endax = oldax.get_xlim()
            oldax.xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
            oldax.xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
            oldax.xaxis.set_label_position('top')
            
            plt.setp(oldax.get_xticklabels(), fontsize=160)
            plt.setp(oldax.get_yticklabels(), fontsize=160)
            plt.setp(oldax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(oldax.yaxis.get_majorticklabels(), fontsize=160)

            newax = plt.gcf().add_axes(oldax.get_position(), sharex=oldax, frameon=False)
            newax.tick_params(axis='x', labelbottom=False, labeltop=True)
            plt.setp(newax.get_xticklabels(), fontsize=160)
            plt.setp(newax.get_yticklabels(), fontsize=160)
            p.plot_marginal(i-1, ls='-', color='blue', linewidth=3)
            newax.set_ylim(0, 1)

            ylim = newax.get_ylim()
            y = ylim[0] + 0.05*(ylim[1] - ylim[0])
            center = m['median']
            low1, high1 = m['1sigma']
            #print(center, low1, high1)
            newax.errorbar(x=center, y=y,
                xerr=np.transpose([[center - low1, high1 - center]]), 
                color='blue', linewidth=2, marker='s')
            oldax.set_yticks([])
            #newax.set_yticks([])
            oldax.tick_params(axis='both', length=30, direction='in', right=True)
            newax.tick_params(axis='both', length=30, direction='in', right=True, pad=30)
            newax.yaxis.set_label_position('right')
            newax.xaxis.set_label_position('top')
            newax.yaxis.set_ticks(np.arange(0.2, 1.2, 0.2))
            newax.tick_params(axis='y', labelleft=False, labelright=True)
            newax.set_ylabel("Probability", fontsize=200, labelpad=30)
            plt.setp(newax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(newax.yaxis.get_majorticklabels(), fontsize=160)
            ylim = oldax.get_ylim()
            newax.set_xlim(m['5sigma'])
            oldax.set_xlim(m['5sigma'])
            #plt.close()
            
            if i == n_params:
                break
                
            #for j in range(i):
            for j in range(params_count2[params_count_indx], n_params-1):
                stepsizes = [0.05, 0.4, 0.35, 10, 20, 40, 4e-8, 0.4]
                axis_formatters = ['%0.2f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
                plt.subplot(n_params, n_params, ((n_params + i) + j*n_params))
                p.plot_conditional(i-1, j+1, bins=20, cmap = pl.cm.gray_r)
                for m in modes:
                    plt.errorbar(x=bestfit_params[i-1], y=bestfit_params[j+1], xerr=m['sigma'][i-1], yerr=m['sigma'][j+1])
                    
                    ########
                if j == 6 and i != 1:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j != 6:
                    plt.ylabel(parameters[j+1], fontsize=200)
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j == 6:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    plt.ylabel(parameters[j+1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                else:   
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
    
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                        
                    plt.gca().axes.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                plt.xticks(rotation=40)
                plt.setp(plt.gca().get_xticklabels(), fontsize=160)
                plt.setp(plt.gca().get_yticklabels(), fontsize=160)
                #plt.savefig('cond_%s_%s.pdf' % (params[i], params[j]), bbox_tight=True)
                #plt.close()
                
            params_count_indx += 1

        plt.savefig(dest + 'mod_a_med_xe_marg'+str(detections[d]['Detection'])+'.pdf')
        #plt.savefig(dest + 'marg'+str(detections[d]['Detection'])+'.png')
        plt.close()
    
    '''
    Storing the average parameter values
    '''
    # Open a text file to save the parameter results
    av_results_txt = open(dest+'mod_a_med_xe_av_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')

    # List to store best parameters
    av_best_par = []
    
    h = 0
    for par in parameters:
        av_results_txt.write('%s %.15f %.15f\n' % (par, modes[0]['mean'][h], modes[0]['sigma'][h]))
        av_best_par.append([modes[0]['mean'][h], modes[0]['sigma'][h]])
        h+=1

    av_results_txt.close()
    mod_a_med_xe_av_best_par_list.append(av_best_par)
    
    '''
    Choosing the median
    '''
    # Open a text file to save the parameter results_txt
    med_results_txt = open(dest+'mod_a_med_xe_med_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    # Write the marginal likelihood
    med_results_txt.write('lnZ %.1f %.1f\n'%(stats['global evidence'], stats['global evidence error']))
    
    # List to store best parameters
    med_best_par = []
    
    # Write the parameters
    for par, marg in zip(parameters, stats['marginals']):
        # 1 sigma provides the lowest and highest estimates
        lo, hi = marg['1sigma']
        sigma = (hi - lo) / 2 # Calculate the error
        
        med = marg['median']
        
        if sigma == 0:
            j = 3
        else:
            j = max(0, int(-np.floor(np.log10(sigma))) + 1)
            
        # Create the string to write    
        fmt = '%%.%df' % j
        fmts = '\t'.join(['%-3s' + fmt + " +- " + fmt +'\n'])
        
        med_results_txt.write('%s %.15f %.15f\n' % (par, med, sigma))
        
        # Save the value and error for this parameter
        med_best_par.append([med, sigma])
    
    med_results_txt.close()
    mod_a_med_xe_med_best_par_list.append(med_best_par)

    '''
    Model and integrated flux
    '''
    # The busy function with the best parameter
    modelx = np.linspace(min(all_vel[d]), max(all_vel[d]), 1000)
    modely = Bmodelx(modelx, med_best_par[0][0], med_best_par[1][0], med_best_par[2][0], med_best_par[3][0], 
                            med_best_par[4][0], med_best_par[5][0], med_best_par[6][0], med_best_par[7][0])
    
    # Integrated flux of data
    Sdv_data = all_sdv[d]
    Sdv_data_err = all_sdv_err[d]
    
    # Integrated flux of model
    dvmodel = np.average(np.diff(modelx))
    Sdv_model = np.sum(modely*dvmodel)*all_max_flux[d]
    
    '''
    Plotting the median and mean parameters
    '''
    pl.figure(figsize=(15,7)) 
    pl.step(all_vel[d], all_flux[d], 'k', where='mid')
    pl.errorbar(all_vel[d], all_flux[d], yerr=all_flux_err[d], fmt='', marker=None, ls='none', color='k')
    pl.plot(modelx, modely, 'r-')
#     pl.plot(modelx, Bmodelx(modelx, av_best_par[0][0], av_best_par[1][0], av_best_par[2][0], av_best_par[3][0], 
#                             av_best_par[4][0], av_best_par[5][0], av_best_par[6][0], av_best_par[7][0]), 'b-', 
#             alpha=0.5, label=r'Mean W$_{50}$ = %.2f $\pm$ %.2f km/s'%(2*av_best_par[3][0], 2*av_best_par[3][1]))
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Axes parameters
    pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
    pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)
    #pl.legend(loc='upper right', fontsize=17)
    
    # Anchored text
    textstr = '\n'.join((
    r'Med. W$_{50}$ = %.2f $\pm$ %.2f km s$^{-1}$' % (2*med_best_par[3][0], 2*med_best_par[3][1], ),
    r'$\int S\mathrm{dv}_\mathrm{data}$ = %.3f $\pm$ %.3f Jy km s$^{-1}$' % (Sdv_data, Sdv_data_err, ),
    r'$\int S\mathrm{dv}_\mathrm{model}$ = %.3f Jy km s$^{-1}$' % (Sdv_model, )))
    pl.annotate(textstr, xy=(0.03, 0.7), xycoords='axes fraction', fontsize=17)
    
    pl.savefig(dest+'mod_a_med_xe_best_par'+str(detections[d]['Detection'])+'.png')
    pl.close()

## Third run with modified priors for $a$, and $x_e$ and $x_p$ fixed on the median

In [ ]:
path_to_data = '/users/wanga/mightee/analysis/COSMOS_1330/masked_profiles_COSMOS_1330_output/'

# Loop through each detection
for i in range(len(detections)):        
    # Define the prior with a different paramater space for a
    # Define the prior that transforms the unit cube into the parameter cube that is now in log space
    # parameters are a, b1, b2, w, xe, xp, c, n
    def prior(cube, ndim, nparams):
        cube[0] = cube[0]*(three_sigma_a[i][1] - three_sigma_a[i][0]) + three_sigma_a[i][0] # modified prior for a within 3 sigma
        cube[1] = cube[1] # log-uniform prior for b1 between 0 and 1
        cube[2] = cube[2] # log-uniform prior for b2 between 0 and 1
        cube[3] = 10**(cube[3]*3) # log-uniform prior for w between 10**(0*3)=1 and 10**(1*3)=1e3
        cube[4] = mod_a_med_xe_med_best_par_list[i][4][0] # xe fixed on medians of second run
        cube[5] = mod_a_med_xe_med_best_par_list[i][5][0] # xp fixed on medians of second run
        cube[6] = 10**(-cube[6]*2 - 7) # log-uniform prior for c between 10**(-0*2-7)=1e-7 and 10**(-1*2-7)=1e-9
        cube[7] = 6*cube[7] + 2 # log-uniform prior for n between 6(0)+2=2 and 6(1)+2=8
    
    # Create a directory for detection i's results
    directory_i = dirName+'/detection_'+str(detections[i]['Detection'])+'/'
    
    vel = all_vel[i]
    fluxdata = all_flux[i]
    fluxerr = all_flux_err[i]
    
    noise = fluxerr
    
    # Number of dimensions our problem has
    parameters = [r'$a$', r'$b_1$', r'$b_2$', r'$w$', r'$x_e$', r'$x_p$', r'$c$', r'$n$']
    n_params = len(parameters)
    
    # Run PyMultiNest
    pymultinest.run(loglike, prior, n_params, outputfiles_basename='mod_a_med_xexp_detection_'+str(detections[i]['Detection']), resume=False, verbose=True)
    json.dump(parameters, open(directory_i+'/mod_a_med_xexp_params.json', 'w')) # Save the parameters
    
    # plot the distribution of a posteriori possible models
    pl.figure(figsize=(15,7)) 
    pl.step(vel, fluxdata, '-', color='black', where='mid')
    pl.errorbar(vel, fluxdata, yerr=fluxerr, fmt='', marker=None, ls='none', color='k')
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Plot the distribution of a posteriori possible models
    # Modelx is just a linear space of the velocities
    modelx = np.linspace(min(vel), max(vel), 1000)

    an = pymultinest.Analyzer(outputfiles_basename='mod_a_med_xexp_detection_'+str(detections[i]['Detection']), n_params = n_params)
    
    # Loop through each possible model and plot it
    for (a, b1, b2, w, xe, xp, c, n) in an.get_equal_weighted_posterior()[::100,:-1]:
        pl.plot(modelx, Bmodelx(modelx, a, b1, b2, w, xe, xp, c, n), '-', color='red', alpha=0.3)
        
        # Axes parameters
        pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
        pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)

    pl.savefig(directory_i+'mod_a_med_xexp_posterior'+str(detections[i]['Detection'])+'.png')
    pl.close()

Moving the output of PyMultiNest into the relevant directories. May need to run this cell twice if errors arise.

In [23]:
source = '/users/wanga/mightee/analysis/COSMOS_1330/'

# Read in all the files in the directory
files = os.listdir(source)

# Detections with two digit numbers confuse python so sort detections in reverse order
rev_det = [detections[i]['Detection'] for i in range(len(detections))][::-1]

# Loop through each detection
for i in rev_det:
    # Path to detection folder
    dest = dirName+'detection_'+str(i) 
    
    # Loop through all the files
    for file in files:
        if (file.startswith('mod_a_med_xexp_detection_'+str(i))):
            shutil.move(file, dest)

In [24]:
# List to store median and average parameters for each profile
mod_a_med_xexp_med_best_par_list = []
mod_a_med_xexp_av_best_par_list = []

In [ ]:
# Create the marginal plots for each detection
for d in range(len(detections)):
    # Path to detection folder
    dest = dirName+'detection_'+str(detections[d]['Detection'])+'/'
    
    # Load the parameters
    params = json.load(open(dest+'mod_a_med_xexp_params.json'))
    n_params = len(params)
    
    # Create the analyzer object
    an = pymultinest.Analyzer(n_params = n_params, outputfiles_basename = dest+'mod_a_med_xexp_detection_'+str(detections[d]['Detection']))
    
    # Get a dictionary containing information about the logZ and its errors,
    # the individual modes and their parameters, quantiles of the parameter posteriors
    #stats = an.get_stats()
    stats = an.get_stats()
    
    # get the best fit (highest likelihood) point
    bestfit_params_dict = an.get_best_fit()
    bestfit_params = bestfit_params_dict.get('parameters')
    
    json.dump(stats, open(dest+'mod_a_med_xexp_stats.json', 'w+'))
    
    # Create the marginal plot
    p = pymultinest.PlotMarginal(an)
    
    values = an.get_equal_weighted_posterior()
    assert n_params == len(stats['marginals'])
    modes = stats['modes']
    
    # Dimensionality of the plot
    dim2 = os.environ.get('D', '1' if n_params > 20 else '2') == '2'
    nbins = 100 if n_params < 3 else 20
    
    # Set up grid layout
    params_count2 = [0, 1, 2, 3, 4, 5, 6] # Grid for j
    params_count_indx = 0
    
    if dim2:
        plt.figure(figsize=(27*n_params, 27*n_params))
        #for i in range(n_params):
        for i in range(1, n_params+1, 1):
            stepsizes = [0.1, 0.35, 0.35, 10, 10, 50, 4e-8, 0.4]
            axis_formatters = ['%0.1f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
            plt.subplot(n_params, n_params, (1 + 9*(i-1)))
            plt.subplots_adjust(hspace=0.4, wspace=0.4)
            
            plt.xlabel(parameters[i-1], fontsize=160)

            m = stats['marginals'][i-1]
            
            plt.xlim(m['5sigma'])
            plt.xticks(rotation=40)

            oldax = plt.gca()
            x,w,patches = oldax.hist(values[:,i-1], bins=nbins, edgecolor='grey', color='grey', histtype='stepfilled', alpha=0.2)
            oldax.set_ylim(0, x.max())
            oldax.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
            oldax.tick_params(axis='x', labelbottom=False, labeltop=True)
            
            startax, endax = oldax.get_xlim()
            oldax.xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
            oldax.xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
            oldax.xaxis.set_label_position('top')
            
            plt.setp(oldax.get_xticklabels(), fontsize=160)
            plt.setp(oldax.get_yticklabels(), fontsize=160)
            plt.setp(oldax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(oldax.yaxis.get_majorticklabels(), fontsize=160)

            newax = plt.gcf().add_axes(oldax.get_position(), sharex=oldax, frameon=False)
            newax.tick_params(axis='x', labelbottom=False, labeltop=True)
            plt.setp(newax.get_xticklabels(), fontsize=160)
            plt.setp(newax.get_yticklabels(), fontsize=160)
            p.plot_marginal(i-1, ls='-', color='blue', linewidth=3)
            newax.set_ylim(0, 1)

            ylim = newax.get_ylim()
            y = ylim[0] + 0.05*(ylim[1] - ylim[0])
            center = m['median']
            low1, high1 = m['1sigma']
            #print(center, low1, high1)
            newax.errorbar(x=center, y=y,
                xerr=np.transpose([[center - low1, high1 - center]]), 
                color='blue', linewidth=2, marker='s')
            oldax.set_yticks([])
            #newax.set_yticks([])
            oldax.tick_params(axis='both', length=30, direction='in', right=True)
            newax.tick_params(axis='both', length=30, direction='in', right=True, pad=30)
            newax.yaxis.set_label_position('right')
            newax.xaxis.set_label_position('top')
            newax.yaxis.set_ticks(np.arange(0.2, 1.2, 0.2))
            newax.tick_params(axis='y', labelleft=False, labelright=True)
            newax.set_ylabel("Probability", fontsize=200, labelpad=30)
            plt.setp(newax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(newax.yaxis.get_majorticklabels(), fontsize=160)
            ylim = oldax.get_ylim()
            newax.set_xlim(m['5sigma'])
            oldax.set_xlim(m['5sigma'])
            #plt.close()
            
            if i == n_params:
                break
                
            #for j in range(i):
            for j in range(params_count2[params_count_indx], n_params-1):
                stepsizes = [0.05, 0.4, 0.35, 10, 20, 40, 4e-8, 0.4]
                axis_formatters = ['%0.2f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
                plt.subplot(n_params, n_params, ((n_params + i) + j*n_params))
                p.plot_conditional(i-1, j+1, bins=20, cmap = pl.cm.gray_r)
                for m in modes:
                    plt.errorbar(x=bestfit_params[i-1], y=bestfit_params[j+1], xerr=m['sigma'][i-1], yerr=m['sigma'][j+1])
                    
                    ########
                if j == 6 and i != 1:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j != 6:
                    plt.ylabel(parameters[j+1], fontsize=200)
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j == 6:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    plt.ylabel(parameters[j+1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                else:   
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
    
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                        
                    plt.gca().axes.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                plt.xticks(rotation=40)
                plt.setp(plt.gca().get_xticklabels(), fontsize=160)
                plt.setp(plt.gca().get_yticklabels(), fontsize=160)
                #plt.savefig('cond_%s_%s.pdf' % (params[i], params[j]), bbox_tight=True)
                #plt.close()
                
            params_count_indx += 1

        plt.savefig(dest + 'mod_a_med_xexp_marg'+str(detections[d]['Detection'])+'.pdf')
        #plt.savefig(dest + 'marg'+str(detections[d]['Detection'])+'.png')
        plt.close()
    
    '''
    Storing the average parameter values
    '''
    # Open a text file to save the parameter results
    av_results_txt = open(dest+'mod_a_med_xexp_av_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')

    # List to store best parameters
    av_best_par = []
    
    h = 0
    for par in parameters:
        av_results_txt.write('%s %.15f %.15f\n' % (par, modes[0]['mean'][h], modes[0]['sigma'][h]))
        av_best_par.append([modes[0]['mean'][h], modes[0]['sigma'][h]])
        h+=1

    av_results_txt.close()
    mod_a_med_xexp_av_best_par_list.append(av_best_par)
    
    '''
    Choosing the median
    '''
    # Open a text file to save the parameter results_txt
    med_results_txt = open(dest+'mod_a_med_xexp_med_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    # Write the marginal likelihood
    med_results_txt.write('lnZ %.1f %.1f\n'%(stats['global evidence'], stats['global evidence error']))
    
    # List to store best parameters
    med_best_par = []
    
    # Write the parameters
    for par, marg in zip(parameters, stats['marginals']):
        # 1 sigma provides the lowest and highest estimates
        lo, hi = marg['1sigma']
        sigma = (hi - lo) / 2 # Calculate the error
        
        med = marg['median']
        
        if sigma == 0:
            j = 3
        else:
            j = max(0, int(-np.floor(np.log10(sigma))) + 1)
            
        # Create the string to write    
        fmt = '%%.%df' % j
        fmts = '\t'.join(['%-3s' + fmt + " +- " + fmt +'\n'])
        
        med_results_txt.write('%s %.15f %.15f\n' % (par, med, sigma))
        
        # Save the value and error for this parameter
        med_best_par.append([med, sigma])
    
    med_results_txt.close()
    mod_a_med_xexp_med_best_par_list.append(med_best_par)
    
    '''
    Model and integrated flux
    '''
    # The busy function with the best parameter
    modelx = np.linspace(min(all_vel[d]), max(all_vel[d]), 1000)
    modely = Bmodelx(modelx, med_best_par[0][0], med_best_par[1][0], med_best_par[2][0], med_best_par[3][0], 
                            med_best_par[4][0], med_best_par[5][0], med_best_par[6][0], med_best_par[7][0])
    
    # Integrated flux of data
    Sdv_data = all_sdv[d]
    Sdv_data_err = all_sdv_err[d]
    
    # Integrated flux of model
    dvmodel = np.average(np.diff(modelx))
    Sdv_model = np.sum(modely*dvmodel)*all_max_flux[d]
    
    '''
    Plotting the median and mean parameters
    '''
    pl.figure(figsize=(15,7)) 
    pl.step(all_vel[d], all_flux[d], 'k', where='mid')
    pl.errorbar(all_vel[d], all_flux[d], yerr=all_flux_err[d], fmt='', marker=None, ls='none', color='k')
    pl.plot(modelx, modely, 'r-')
#     pl.plot(modelx, Bmodelx(modelx, av_best_par[0][0], av_best_par[1][0], av_best_par[2][0], av_best_par[3][0], 
#                             av_best_par[4][0], av_best_par[5][0], av_best_par[6][0], av_best_par[7][0]), 'b-', 
#             alpha=0.5, label=r'Mean W$_{50}$ = %.2f $\pm$ %.2f km/s'%(2*av_best_par[3][0], 2*av_best_par[3][1]))
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Axes parameters
    pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
    pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)
    #pl.legend(loc='upper right', fontsize=17)
    
    # Anchored text
    textstr = '\n'.join((
    r'Med. W$_{50}$ = %.2f $\pm$ %.2f km s$^{-1}$' % (2*med_best_par[3][0], 2*med_best_par[3][1], ),
    r'$\int S\mathrm{dv}_\mathrm{data}$ = %.3f $\pm$ %.3f Jy km s$^{-1}$' % (Sdv_data, Sdv_data_err, ),
    r'$\int S\mathrm{dv}_\mathrm{model}$ = %.3f Jy km s$^{-1}$' % (Sdv_model, )))
    pl.annotate(textstr, xy=(0.03, 0.7), xycoords='axes fraction', fontsize=17)
    
    pl.savefig(dest+'mod_a_med_xexp_best_par'+str(detections[d]['Detection'])+'.png')
    pl.close()

## Fourth run with $x_e$, $x_p$, $b_1$ and $b_2$ fixed on the median, and the ranges for $a$ relaxed 

In [ ]:
path_to_data = '/users/wanga/mightee/analysis/COSMOS_1330/masked_profiles_COSMOS_1330_output/'

# Loop through each detection
for i in range(len(detections)):        
    # Define the prior with a different paramater space for a
    # Define the prior that transforms the unit cube into the parameter cube that is now in log space
    # parameters are a, b1, b2, w, xe, xp, c, n
    def prior(cube, ndim, nparams):
        cube[0] = cube[0] # log-uniform prior for a between 0*1=0 and 1*1=1
        cube[1] = mod_a_med_xexp_med_best_par_list[i][1][0] # b1 fixed on medians of third run
        cube[2] = mod_a_med_xexp_med_best_par_list[i][2][0] # b2 fixed on medians of third run
        cube[3] = 10**(cube[3]*3) # log-uniform prior for w between 10**(0*3)=1 and 10**(1*3)=1e3
        cube[4] = mod_a_med_xexp_med_best_par_list[i][4][0] # xe fixed on medians of third run
        cube[5] = mod_a_med_xexp_med_best_par_list[i][5][0] # xp fixed on medians of third run
        cube[6] = 10**(-cube[6]*2 - 7) # log-uniform prior for c between 10**(-0*2-7)=1e-7 and 10**(-1*2-7)=1e-9
        cube[7] = 6*cube[7] + 2 # log-uniform prior for n between 6(0)+2=2 and 6(1)+2=8
    
    # Create a directory for detection i's results
    directory_i = dirName+'/detection_'+str(detections[i]['Detection'])+'/'
    
    vel = all_vel[i]
    fluxdata = all_flux[i]
    fluxerr = all_flux_err[i]
    
    noise = fluxerr
    
    # Number of dimensions our problem has
    parameters = [r'$a$', r'$b_1$', r'$b_2$', r'$w$', r'$x_e$', r'$x_p$', r'$c$', r'$n$']
    n_params = len(parameters)
    
    # Run PyMultiNest
    pymultinest.run(loglike, prior, n_params, outputfiles_basename='med_xexpb1b2_detection_'+str(detections[i]['Detection']), resume=False, verbose=True)
    json.dump(parameters, open(directory_i+'/med_xexpb1b2_params.json', 'w')) # Save the parameters
    
    # plot the distribution of a posteriori possible models
    pl.figure(figsize=(15,7)) 
    pl.step(vel, fluxdata, '-', color='black', where='mid')
    pl.errorbar(vel, fluxdata, yerr=fluxerr, fmt='', marker=None, ls='none', color='k')
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Plot the distribution of a posteriori possible models
    # Modelx is just a linear space of the velocities
    modelx = np.linspace(min(vel), max(vel), 1000)

    an = pymultinest.Analyzer(outputfiles_basename='med_xexpb1b2_detection_'+str(detections[i]['Detection']), n_params = n_params)
    
    # Loop through each possible model and plot it
    for (a, b1, b2, w, xe, xp, c, n) in an.get_equal_weighted_posterior()[::100,:-1]:
        pl.plot(modelx, Bmodelx(modelx, a, b1, b2, w, xe, xp, c, n), '-', color='red', alpha=0.3)
        
        # Axes parameters
        pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
        pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
        pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)

    pl.savefig(directory_i+'med_xexpb1b2_posterior'+str(detections[i]['Detection'])+'.png')
    pl.close()

Moving the output of PyMultiNest into the relevant directories. May need to run this cell twice if errors arise.

In [28]:
source = '/users/wanga/mightee/analysis/COSMOS_1330/'

# Read in all the files in the directory
files = os.listdir(source)

# Detections with two digit numbers confuse python so sort detections in reverse order
rev_det = [detections[i]['Detection'] for i in range(len(detections))][::-1]

# Loop through each detection
for i in rev_det:
    # Path to detection folder
    dest = dirName+'detection_'+str(i) 
    
    # Loop through all the files
    for file in files:
        if (file.startswith('med_xexpb1b2_detection_'+str(i))):
            shutil.move(file, dest)

In [29]:
# List to store median and average parameters for each profile
med_xexpb1b2_med_best_par_list = []
med_xexpb1b2_av_best_par_list = []

In [ ]:
# Create the marginal plots for each detection
for d in range(len(detections)):
    # Path to detection folder
    dest = dirName+'detection_'+str(detections[d]['Detection'])+'/'
    
    # Load the parameters
    params = json.load(open(dest+'med_xexpb1b2_params.json'))
    n_params = len(params)
    
    # Create the analyzer object
    an = pymultinest.Analyzer(n_params = n_params, outputfiles_basename = dest+'med_xexpb1b2_detection_'+str(detections[d]['Detection']))
    
    # Get a dictionary containing information about the logZ and its errors,
    # the individual modes and their parameters, quantiles of the parameter posteriors
    #stats = an.get_stats()
    stats = an.get_stats()
    
    # get the best fit (highest likelihood) point
    bestfit_params_dict = an.get_best_fit()
    bestfit_params = bestfit_params_dict.get('parameters')
    
    json.dump(stats, open(dest+'med_xexpb1b2_stats.json', 'w+'))
    
    # Create the marginal plot
    p = pymultinest.PlotMarginal(an)
    
    values = an.get_equal_weighted_posterior()
    assert n_params == len(stats['marginals'])
    modes = stats['modes']
    
    # Dimensionality of the plot
    dim2 = os.environ.get('D', '1' if n_params > 20 else '2') == '2'
    nbins = 100 if n_params < 3 else 20
    
    # Set up grid layout
    params_count2 = [0, 1, 2, 3, 4, 5, 6] # Grid for j
    params_count_indx = 0
    
    if dim2:
        plt.figure(figsize=(27*n_params, 27*n_params))
        #for i in range(n_params):
        for i in range(1, n_params+1, 1):
            stepsizes = [0.1, 0.35, 0.35, 10, 10, 50, 4e-8, 0.4]
            axis_formatters = ['%0.1f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
            plt.subplot(n_params, n_params, (1 + 9*(i-1)))
            plt.subplots_adjust(hspace=0.4, wspace=0.4)
            
            plt.xlabel(parameters[i-1], fontsize=160)

            m = stats['marginals'][i-1]
            
            plt.xlim(m['5sigma'])
            plt.xticks(rotation=40)

            oldax = plt.gca()
            x,w,patches = oldax.hist(values[:,i-1], bins=nbins, edgecolor='grey', color='grey', histtype='stepfilled', alpha=0.2)
            oldax.set_ylim(0, x.max())
            oldax.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
            oldax.tick_params(axis='x', labelbottom=False, labeltop=True)
            
            startax, endax = oldax.get_xlim()
            oldax.xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
            oldax.xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
            oldax.xaxis.set_label_position('top')
            
            plt.setp(oldax.get_xticklabels(), fontsize=160)
            plt.setp(oldax.get_yticklabels(), fontsize=160)
            plt.setp(oldax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(oldax.yaxis.get_majorticklabels(), fontsize=160)

            newax = plt.gcf().add_axes(oldax.get_position(), sharex=oldax, frameon=False)
            newax.tick_params(axis='x', labelbottom=False, labeltop=True)
            plt.setp(newax.get_xticklabels(), fontsize=160)
            plt.setp(newax.get_yticklabels(), fontsize=160)
            p.plot_marginal(i-1, ls='-', color='blue', linewidth=3)
            newax.set_ylim(0, 1)

            ylim = newax.get_ylim()
            y = ylim[0] + 0.05*(ylim[1] - ylim[0])
            center = m['median']
            low1, high1 = m['1sigma']
            #print(center, low1, high1)
            newax.errorbar(x=center, y=y,
                xerr=np.transpose([[center - low1, high1 - center]]), 
                color='blue', linewidth=2, marker='s')
            oldax.set_yticks([])
            #newax.set_yticks([])
            oldax.tick_params(axis='both', length=30, direction='in', right=True)
            newax.tick_params(axis='both', length=30, direction='in', right=True, pad=30)
            newax.yaxis.set_label_position('right')
            newax.xaxis.set_label_position('top')
            newax.yaxis.set_ticks(np.arange(0.2, 1.2, 0.2))
            newax.tick_params(axis='y', labelleft=False, labelright=True)
            newax.set_ylabel("Probability", fontsize=200, labelpad=30)
            plt.setp(newax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(newax.yaxis.get_majorticklabels(), fontsize=160)
            ylim = oldax.get_ylim()
            newax.set_xlim(m['5sigma'])
            oldax.set_xlim(m['5sigma'])
            #plt.close()
            
            if i == n_params:
                break
                
            #for j in range(i):
            for j in range(params_count2[params_count_indx], n_params-1):
                stepsizes = [0.05, 0.4, 0.35, 10, 20, 40, 4e-8, 0.4]
                axis_formatters = ['%0.2f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
                plt.subplot(n_params, n_params, ((n_params + i) + j*n_params))
                p.plot_conditional(i-1, j+1, bins=20, cmap = pl.cm.gray_r)
                for m in modes:
                    plt.errorbar(x=bestfit_params[i-1], y=bestfit_params[j+1], xerr=m['sigma'][i-1], yerr=m['sigma'][j+1])
                    
                    ########
                if j == 6 and i != 1:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j != 6:
                    plt.ylabel(parameters[j+1], fontsize=200)
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j == 6:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    plt.ylabel(parameters[j+1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                else:   
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
    
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                        
                    plt.gca().axes.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                plt.xticks(rotation=40)
                plt.setp(plt.gca().get_xticklabels(), fontsize=160)
                plt.setp(plt.gca().get_yticklabels(), fontsize=160)
                #plt.savefig('cond_%s_%s.pdf' % (params[i], params[j]), bbox_tight=True)
                #plt.close()
                
            params_count_indx += 1

        plt.savefig(dest + 'med_xexpb1b2_marg'+str(detections[d]['Detection'])+'.pdf')
        #plt.savefig(dest + 'marg'+str(detections[d]['Detection'])+'.png')
        plt.close()
    
    '''
    Storing the average parameter values
    '''
    # Open a text file to save the parameter results
    av_results_txt = open(dest+'med_xexpb1b2_av_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')

    # List to store best parameters
    av_best_par = []
    
    h = 0
    for par in parameters:
        av_results_txt.write('%s %.15f %.15f\n' % (par, modes[0]['mean'][h], modes[0]['sigma'][h]))
        av_best_par.append([modes[0]['mean'][h], modes[0]['sigma'][h]])
        h+=1

    av_results_txt.close()
    med_xexpb1b2_av_best_par_list.append(av_best_par)
    
    '''
    Choosing the median
    '''
    # Open a text file to save the parameter results_txt
    med_results_txt = open(dest+'med_xexpb1b2_med_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    # Write the marginal likelihood
    med_results_txt.write('lnZ %.1f %.1f\n'%(stats['global evidence'], stats['global evidence error']))
    
    # List to store best parameters
    med_best_par = []
    
    # Write the parameters
    for par, marg in zip(parameters, stats['marginals']):
        # 1 sigma provides the lowest and highest estimates
        lo, hi = marg['1sigma']
        sigma = (hi - lo) / 2 # Calculate the error
        
        med = marg['median']
        
        if sigma == 0:
            j = 3
        else:
            j = max(0, int(-np.floor(np.log10(sigma))) + 1)
            
        # Create the string to write    
        fmt = '%%.%df' % j
        fmts = '\t'.join(['%-3s' + fmt + " +- " + fmt +'\n'])
        
        med_results_txt.write('%s %.15f %.15f\n' % (par, med, sigma))
        
        # Save the value and error for this parameter
        med_best_par.append([med, sigma])
    
    med_results_txt.close()
    med_xexpb1b2_med_best_par_list.append(med_best_par)
    
    '''
    Model and integrated flux
    '''
    # The busy function with the best parameter
    modelx = np.linspace(min(all_vel[d]), max(all_vel[d]), 1000)
    modely = Bmodelx(modelx, med_best_par[0][0], med_best_par[1][0], med_best_par[2][0], med_best_par[3][0], 
                            med_best_par[4][0], med_best_par[5][0], med_best_par[6][0], med_best_par[7][0])
    
    # Integrated flux of data
    Sdv_data = all_sdv[d]
    Sdv_data_err = all_sdv_err[d]
    
    # Integrated flux of model
    dvmodel = np.average(np.diff(modelx))
    Sdv_model = np.sum(modely*dvmodel)*all_max_flux[d]
    
    '''
    Plotting the median and mean parameters
    '''
    pl.figure(figsize=(15,7)) 
    pl.step(all_vel[d], all_flux[d], 'k', where='mid')
    pl.errorbar(all_vel[d], all_flux[d], yerr=all_flux_err[d], fmt='', marker=None, ls='none', color='k')
    pl.plot(modelx, modely, 'r-')
#     pl.plot(modelx, Bmodelx(modelx, av_best_par[0][0], av_best_par[1][0], av_best_par[2][0], av_best_par[3][0], 
#                             av_best_par[4][0], av_best_par[5][0], av_best_par[6][0], av_best_par[7][0]), 'b-', 
#             alpha=0.5, label=r'Mean W$_{50}$ = %.2f $\pm$ %.2f km/s'%(2*av_best_par[3][0], 2*av_best_par[3][1]))
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Axes parameters
    pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
    pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)
    #pl.legend(loc='upper right', fontsize=17)
    
    # Anchored text
    textstr = '\n'.join((
    r'Med. W$_{50}$ = %.2f $\pm$ %.2f km s$^{-1}$' % (2*med_best_par[3][0], 2*med_best_par[3][1], ),
    r'$\int S\mathrm{dv}_\mathrm{data}$ = %.3f $\pm$ %.3f Jy km s$^{-1}$' % (Sdv_data, Sdv_data_err, ),
    r'$\int S\mathrm{dv}_\mathrm{model}$ = %.3f Jy km s$^{-1}$' % (Sdv_model, )))
    pl.annotate(textstr, xy=(0.03, 0.7), xycoords='axes fraction', fontsize=17)
    
    pl.savefig(dest+'med_xexpb1b2_best_par'+str(detections[d]['Detection'])+'.png')
    pl.close()

## Fifth run with $b_1$, $b_2$, $x_e$, $x_p$ and $n$ fixed on the median

In [31]:
# List to store paramaters of fit with integrated flux that best matches data
chi_best_fits = []

In [32]:
# List to store integrated fluxes
sdv_models = []
sdv_sampled_models = []

In [ ]:
path_to_data = '/users/wanga/mightee/analysis/COSMOS_1330/masked_profiles_COSMOS_1330_output/'

# Loop through each detection
for i in range(len(detections)): 
    # Define the prior with a different paramater space for a
    # Define the prior that transforms the unit cube into the parameter cube that is now in log space
    # parameters are a, b1, b2, w, xe, xp, c, n
    def prior(cube, ndim, nparams):
        cube[0] = cube[0] # log-uniform prior for a between 0*1=0 and 1*1=1
        cube[1] = med_xexpb1b2_med_best_par_list[i][1][0] # b1 fixed on medians of fourth run
        cube[2] = med_xexpb1b2_med_best_par_list[i][2][0] # b2 fixed on medians of fourth run
        cube[3] = 10**(cube[3]*3) # log-uniform prior for w between 10**(0*3)=1 and 10**(1*3)=1e3
        cube[4] = med_xexpb1b2_med_best_par_list[i][4][0] # xe fixed on medians of fourth run
        cube[5] = med_xexpb1b2_med_best_par_list[i][5][0] # xp fixed on medians of fourth run
        cube[6] = 10**(-cube[6]*2 - 7) # log-uniform prior for c between 10**(-0*2-7)=1e-7 and 10**(-1*2-7)=1e-9
        cube[7] = med_xexpb1b2_med_best_par_list[i][7][0] # n fixed on medians of fourth run
    
    # Create a directory for detection i's results
    directory_i = dirName+'/detection_'+str(detections[i]['Detection'])+'/'
    
    vel = all_vel[i]
    fluxdata = all_flux[i]
    fluxerr = all_flux_err[i]
    
    noise = fluxerr
    
    # Number of dimensions our problem has
    parameters = [r'$a$', r'$b_1$', r'$b_2$', r'$w$', r'$x_e$', r'$x_p$', r'$c$', r'$n$']
    n_params = len(parameters)
    
    # Run PyMultiNest
    pymultinest.run(loglike, prior, n_params, outputfiles_basename='med_b1b2xexpn_detection_'+str(detections[i]['Detection']), resume=False, verbose=True)
    json.dump(parameters, open(directory_i+'/med_b1b2xexpn_params.json', 'w')) # Save the parameters
    
#     # plot the distribution of a posteriori possible models
#     pl.figure(figsize=(15,7)) 
#     pl.step(vel, fluxdata, '-', color='black', where='mid')
#     pl.errorbar(vel, fluxdata, yerr=fluxerr, fmt='', marker=None, ls='none', color='k')
#     pl.xlabel('Velocity (km/s)', fontsize=20)
#     pl.ylabel('Normalized Flux Density', fontsize=20)
    
    # Plot the distribution of a posteriori possible models
    # Modelx is just a linear space of the velocities
    modelx = np.linspace(min(vel), max(vel), 1000)

    an = pymultinest.Analyzer(outputfiles_basename='med_b1b2xexpn_detection_'+str(detections[i]['Detection']), n_params = n_params)
    
    # Integrated flux of data
    Sdv_data = all_sdv[i]
    Sdv_data_err = all_sdv_err[i]

    # Counter
    fit_count = 0
    
    # List to store the integrated fluxes
    det_sdv_fits = []
    
    # Previous sdv, dummy variable
    # Want to store sdv of fit that best matches data
    prev_sdv = 100
    prev_sdv_44 = 100
    
    det_chi_fits = []
    
    # Loop through each possible model and plot it
    for (a, b1, b2, w, xe, xp, c, n) in an.get_equal_weighted_posterior()[::100,:-1]:
        # Store first three fits for the average
        if fit_count < 4:
            det_chi_fits.append([a, b1, b2, w, xe, xp, c, n])

        modely = Bmodelx(modelx, a, b1, b2, w, xe, xp, c, n)
        
        # Sample the model at every 44 km/s to simulate the resolution of MIGHTEE 4k data
        res = -1*np.average(np.diff(np.array(vel))) # Size of the channel
        diff = modelx[1] - modelx[0] # Initial resolution of the linear space
        dchan = int(res/diff) # Step size for the loop
        obs_vel_44kms = [] # List to store the sampled velocity with minimum resolution of 44 km/s
        obs_flux_44kms = [] # List to store corresponding fluxes

        # Loop through the fluxes and velocities
        for dat in range(0, len(modelx), dchan):
            obs_vel_44kms.append(modelx[dat])
            obs_flux_44kms.append(modely[dat])

        # Integrated flux of model 44 km/s res
        dvmodel_44 = np.average(np.diff(np.array(obs_vel_44kms)))#np.average(np.diff(modelx))
        Sdv_model_44 = np.sum(np.array(obs_flux_44kms)*dvmodel_44)*all_max_flux[i]#np.sum(modely*dvmodel)*all_max_flux[d]
        # Compare to data sdv
        sdv_diff_44 = np.absolute(Sdv_data - Sdv_model_44)
        if prev_sdv_44 > sdv_diff_44:
            prev_sdv_44 = sdv_diff_44
            sdv_count_44 = fit_count
            final_sdv_44 = Sdv_model_44
            #print('sdv_44 match', [a, b1, b2, w, xe, xp, c, n])
        
        # Integrated flux of model not sampled
        dvmodel = np.average(np.diff(modelx))
        Sdv_model = np.sum(modely*dvmodel)*all_max_flux[i]
        # Compare to data sdv
        sdv_diff = np.absolute(Sdv_data - Sdv_model)
        if prev_sdv > sdv_diff:
            prev_sdv = sdv_diff
            sdv_count = fit_count
            final_sdv = Sdv_model
        
        det_sdv_fits.append(Sdv_model_44)
        #print(str(Sdv_data)+'+/-'+str(Sdv_data_err)+', '+str(Sdv_model_44)+', '+str(Sdv_model))

#         # Calculate the chi squared per dof
#         O = fluxdata
#         C = Bmodelx(vel, a, b1, b2, w, xe, xp, c, n)
#         chi_sqr = np.sum(pow(O - C,2))
#         print(chi_sqr)

        fit_count += 1

    # Store averages in list
    chi_best_fits.append([np.average(np.array([i[0] for i in det_chi_fits])),
                         np.average(np.array([i[1] for i in det_chi_fits])),
                         np.average(np.array([i[2] for i in det_chi_fits])),
                         np.average(np.array([i[3] for i in det_chi_fits])),
                         np.average(np.array([i[4] for i in det_chi_fits])),
                         np.average(np.array([i[5] for i in det_chi_fits])),
                         np.average(np.array([i[6] for i in det_chi_fits])),
                         np.average(np.array([i[7] for i in det_chi_fits]))])

    # Counter variable
    count2 = 0
    
    # Plot the posteriors
    pl.figure(figsize=(15,7)) 
    pl.step(vel, fluxdata, '-', color='black', where='mid')
    pl.errorbar(vel, fluxdata, yerr=fluxerr, fmt='', marker=None, ls='none', color='k')
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Axes parameters
    pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
    pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)
    
    for (a, b1, b2, w, xe, xp, c, n) in an.get_equal_weighted_posterior()[::100,:-1]:
        modely = Bmodelx(modelx, a, b1, b2, w, xe, xp, c, n)
        
        # Fit with sdv matching data
        if count2 == sdv_count:
            pl.plot(modelx, modely, '-', color='blue', alpha=0.6, label=r'$\int S\mathrm{dv}$ match from non-sampled fit')
            sdv_models.append([a, b1, b2, w, xe, xp, c, n, final_sdv])
        
        if count2 == sdv_count_44:
            pl.plot(modelx, modely, '-', color='green', alpha=0.6, label=r'$\int S\mathrm{dv}$ match from sampled fit')
            sdv_sampled_models.append([a, b1, b2, w, xe, xp, c, n, final_sdv_44])
            #print('sdv_44 match post', [a, b1, b2, w, xe, xp, c, n])
            
        else:
            pl.plot(modelx, modely, '-', color='red', alpha=0.2)
        
        count2 += 1
        
    pl.legend(loc='upper left', frameon=False, fontsize=15)
    pl.savefig(directory_i+'med_b1b2xexpn_posterior'+str(detections[i]['Detection'])+'.png')
    pl.close()

Moving the output of PyMultiNest into the relevant directories. May need to run this cell twice if errors arise.

In [35]:
source = '/users/wanga/mightee/analysis/COSMOS_1330/'

# Read in all the files in the directory
files = os.listdir(source)

# Detections with two digit numbers confuse python so sort detections in reverse order
rev_det = [detections[i]['Detection'] for i in range(len(detections))][::-1]

# Loop through each detection
for i in rev_det:
    # Path to detection folder
    dest = dirName+'detection_'+str(i) 
    
    # Loop through all the files
    for file in files:
        if (file.startswith('med_b1b2xexpn_detection_'+str(i))):
            shutil.move(file, dest)

In [36]:
# List to store maximum likelihood best parameters
max_likelihood_list = []

# List to store median and average parameters for each profile
med_b1b2xexpn_med_best_par_list = []
med_b1b2xexpn_av_best_par_list = []

In [ ]:
# Create the marginal plots for each detection
for d in range(len(detections)):
    # Path to detection folder
    dest = dirName+'detection_'+str(detections[d]['Detection'])+'/'
    
    # Load the parameters
    params = json.load(open(dest+'med_b1b2xexpn_params.json'))
    n_params = len(params)
    
    # Create the analyzer object
    an = pymultinest.Analyzer(n_params = n_params, outputfiles_basename = dest+'med_b1b2xexpn_detection_'+str(detections[d]['Detection']))
    
    # Get a dictionary containing information about the logZ and its errors,
    # the individual modes and their parameters, quantiles of the parameter posteriors
    #stats = an.get_stats()
    stats = an.get_stats()
    
    # get the best fit (highest likelihood) point
    bestfit_params_dict = an.get_best_fit()
    bestfit_params = bestfit_params_dict.get('parameters')
    
    json.dump(stats, open(dest+'med_b1b2xexpn_stats.json', 'w+'))
    
    # Create the marginal plot
    p = pymultinest.PlotMarginal(an)
    
    values = an.get_equal_weighted_posterior()
    assert n_params == len(stats['marginals'])
    modes = stats['modes']
    
    # Dimensionality of the plot
    dim2 = os.environ.get('D', '1' if n_params > 20 else '2') == '2'
    nbins = 100 if n_params < 3 else 20
    
    # Set up grid layout
    params_count2 = [0, 1, 2, 3, 4, 5, 6] # Grid for j
    params_count_indx = 0
    
    if dim2:
        plt.figure(figsize=(27*n_params, 27*n_params))
        #for i in range(n_params):
        for i in range(1, n_params+1, 1):
            stepsizes = [0.1, 0.35, 0.35, 10, 10, 50, 4e-8, 0.4]
            axis_formatters = ['%0.1f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
            plt.subplot(n_params, n_params, (1 + 9*(i-1)))
            plt.subplots_adjust(hspace=0.4, wspace=0.4)
            
            plt.xlabel(parameters[i-1], fontsize=160)

            m = stats['marginals'][i-1]
            
            plt.xlim(m['5sigma'])
            plt.xticks(rotation=40)

            oldax = plt.gca()
            x,w,patches = oldax.hist(values[:,i-1], bins=nbins, edgecolor='grey', color='grey', histtype='stepfilled', alpha=0.2)
            oldax.set_ylim(0, x.max())
            oldax.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
            oldax.tick_params(axis='x', labelbottom=False, labeltop=True)
            
            startax, endax = oldax.get_xlim()
            oldax.xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
            oldax.xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
            oldax.xaxis.set_label_position('top')
            
            plt.setp(oldax.get_xticklabels(), fontsize=160)
            plt.setp(oldax.get_yticklabels(), fontsize=160)
            plt.setp(oldax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(oldax.yaxis.get_majorticklabels(), fontsize=160)

            newax = plt.gcf().add_axes(oldax.get_position(), sharex=oldax, frameon=False)
            newax.tick_params(axis='x', labelbottom=False, labeltop=True)
            plt.setp(newax.get_xticklabels(), fontsize=160)
            plt.setp(newax.get_yticklabels(), fontsize=160)
            p.plot_marginal(i-1, ls='-', color='blue', linewidth=3)
            newax.set_ylim(0, 1)

            ylim = newax.get_ylim()
            y = ylim[0] + 0.05*(ylim[1] - ylim[0])
            center = m['median']
            low1, high1 = m['1sigma']
            #print(center, low1, high1)
            newax.errorbar(x=center, y=y,
                xerr=np.transpose([[center - low1, high1 - center]]), 
                color='blue', linewidth=2, marker='s')
            oldax.set_yticks([])
            #newax.set_yticks([])
            oldax.tick_params(axis='both', length=30, direction='in', right=True)
            newax.tick_params(axis='both', length=30, direction='in', right=True, pad=30)
            newax.yaxis.set_label_position('right')
            newax.xaxis.set_label_position('top')
            newax.yaxis.set_ticks(np.arange(0.2, 1.2, 0.2))
            newax.tick_params(axis='y', labelleft=False, labelright=True)
            newax.set_ylabel("Probability", fontsize=200, labelpad=30)
            plt.setp(newax.xaxis.get_majorticklabels(), rotation=40, fontsize=160)
            plt.setp(newax.yaxis.get_majorticklabels(), fontsize=160)
            ylim = oldax.get_ylim()
            newax.set_xlim(m['5sigma'])
            oldax.set_xlim(m['5sigma'])
            #plt.close()
            
            if i == n_params:
                break
                
            #for j in range(i):
            for j in range(params_count2[params_count_indx], n_params-1):
                stepsizes = [0.05, 0.4, 0.35, 10, 20, 40, 4e-8, 0.4]
                axis_formatters = ['%0.2f', '%0.1f', '%0.1f', '%d', '%d', '%d', '%.1e', '%0.1f']
            
                plt.subplot(n_params, n_params, ((n_params + i) + j*n_params))
                p.plot_conditional(i-1, j+1, bins=20, cmap = pl.cm.gray_r)
                for m in modes:
                    plt.errorbar(x=bestfit_params[i-1], y=bestfit_params[j+1], xerr=m['sigma'][i-1], yerr=m['sigma'][j+1])
                    
                    ########
                if j == 6 and i != 1:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j != 6:
                    plt.ylabel(parameters[j+1], fontsize=200)
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                elif i == 1 and j == 6:
                    plt.xlabel(parameters[i-1], fontsize=200)
                    plt.ylabel(parameters[j+1], fontsize=200)
                    
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
        
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                    
                    plt.gca().axes.tick_params(axis='x', labelbottom=True)
                    plt.gca().axes.tick_params(axis='y', labelleft=True)
                    plt.gca().axes.tick_params(axis='both', length=20, direction='in', top=True, right=True, pad=30)
                else:   
                    # x axis
                    startax, endax = plt.gca().get_xlim()
                    plt.gca().xaxis.set_ticks(np.arange(startax, endax, stepsizes[i-1]))
                    plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[i-1])) 
    
                    # y axis
                    startay, enday = plt.gca().get_ylim()
                    plt.gca().yaxis.set_ticks(np.arange(startay, enday, stepsizes[j+1]))
                    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter(axis_formatters[j+1])) 
                        
                    plt.gca().axes.tick_params(axis='both', length=30, direction='in', top=True, right=True, pad=30)
                    plt.gca().axes.tick_params(axis='x', labelbottom=False)
                    plt.gca().axes.tick_params(axis='y', labelleft=False)
                plt.xticks(rotation=40)
                plt.setp(plt.gca().get_xticklabels(), fontsize=160)
                plt.setp(plt.gca().get_yticklabels(), fontsize=160)
                #plt.savefig('cond_%s_%s.pdf' % (params[i], params[j]), bbox_tight=True)
                #plt.close()
                
            params_count_indx += 1

        plt.savefig(dest + 'med_b1b2xexpn_marg'+str(detections[d]['Detection'])+'.pdf')
        #plt.savefig(dest + 'marg'+str(detections[d]['Detection'])+'.png')
        plt.close()
        
    '''
    Storing the best fits based on the maximum likelihood
    '''
    # Open a text file to save the parameter results
    maxlik_results_txt = open(dest+'med_b1b2xexpn_maxlik_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    h = 0
    for par in parameters:
        maxlik_results_txt.write('%s %.15f %.15f\n' % (par, bestfit_params[h], 0))
        h+=1

    maxlik_results_txt.close()
    max_likelihood_list.append(bestfit_params)
        
    '''
    Storing the best fits based on the integrated fluxes
    '''
    # Open a text file to save the parameter results
    sdv_results_txt = open(dest+'med_b1b2xexpn_sdv_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    h = 0
    for par in parameters:
        sdv_results_txt.write('%s %.15f %.15f %.15f %.15f\n' % (par, sdv_models[d][h], 0, sdv_sampled_models[d][h], 0))
        h+=1
    sdv_results_txt.write('sdv %.15f %.15f %.15f %.15f\n' % (sdv_models[d][8], 0, sdv_sampled_models[d][8], 0))
    sdv_results_txt.close()
    
    '''
    Storing the average parameter values
    '''
    # Open a text file to save the parameter results
    av_results_txt = open(dest+'med_b1b2xexpn_av_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')

    # List to store best parameters
    av_best_par = []
    
    h = 0
    for par in parameters:
#         av_results_txt.write('%s %.15f %.15f\n' % (par, modes[0]['mean'][h], modes[0]['sigma'][h]))
#         av_best_par.append([modes[0]['mean'][h], modes[0]['sigma'][h]])
        av_results_txt.write('%s %.15f %.15f\n' % (par, chi_best_fits[d][h], 0))
        av_best_par.append([chi_best_fits[d][h], 0])
        h+=1

    av_results_txt.close()
    med_b1b2xexpn_av_best_par_list.append(av_best_par)
    
    '''
    Choosing the median
    '''
    # Open a text file to save the parameter results_txt
    med_results_txt = open(dest+'med_b1b2xexpn_med_results_detection_'+str(detections[d]['Detection'])+'.txt', 'w+')
    
    # Write the marginal likelihood
    med_results_txt.write('lnZ %.1f %.1f\n'%(stats['global evidence'], stats['global evidence error']))
    
    # List to store best parameters
    med_best_par = []
    
    # Write the parameters
    for par, marg in zip(parameters, stats['marginals']):
        # 1 sigma provides the lowest and highest estimates
        lo, hi = marg['1sigma']
        sigma = (hi - lo) / 2 # Calculate the error
        
        med = marg['median']
        
        if sigma == 0:
            j = 3
        else:
            j = max(0, int(-np.floor(np.log10(sigma))) + 1)
            
        # Create the string to write    
        fmt = '%%.%df' % j
        fmts = '\t'.join(['%-3s' + fmt + " +- " + fmt +'\n'])
        
        med_results_txt.write('%s %.15f %.15f\n' % (par, med, sigma))
        
        # Save the value and error for this parameter
        med_best_par.append([med, sigma])
    
    med_results_txt.close()
    med_b1b2xexpn_med_best_par_list.append(med_best_par)
    
    '''
    Model x values
    '''
    # The busy function with the best parameter
    modelx = np.linspace(min(all_vel[d]), max(all_vel[d]), 1000)
    dvmodel = np.average(np.diff(np.array(modelx)))
    
    '''
    Plotting 
    '''
    pl.figure(figsize=(15,7)) 
    # Data
    # Integrated flux of data
    Sdv_data = all_sdv[d]
    Sdv_data_err = all_sdv_err[d]
    pl.step(all_vel[d], all_flux[d], 'k', where='mid', label=r'$\int S\mathrm{dv}_\mathrm{data}$ = %.3f $\pm$ %.3f Jy km s$^{-1}$' % (Sdv_data, Sdv_data_err))
    pl.errorbar(all_vel[d], all_flux[d], yerr=all_flux_err[d], fmt='', marker=None, ls='none', color='k')
    
    # Maximum likelihood
    modely_maxlik = Bmodelx(modelx, bestfit_params[0], bestfit_params[1], bestfit_params[2], bestfit_params[3], 
                            bestfit_params[4], bestfit_params[5], bestfit_params[6], bestfit_params[7])
    # Integrated flux of maximum likelihood model
    Sdv_model_maxlik = np.sum(modely_maxlik*dvmodel)*all_max_flux[d]
    pl.plot(modelx, modely_maxlik, '--', color='fuchsia', alpha=0.7, label=r'$\int S\mathrm{dv}_\mathrm{max. \ likelihood}$ = %.3f Jy km s$^{-1}$' % (Sdv_model_maxlik))
    
    # Median
    modely_med = Bmodelx(modelx, med_best_par[0][0], med_best_par[1][0], med_best_par[2][0], med_best_par[3][0], 
                            med_best_par[4][0], med_best_par[5][0], med_best_par[6][0], med_best_par[7][0])
    # Integrated flux of median model
    Sdv_model_med = np.sum(modely_med*dvmodel)*all_max_flux[d]
    pl.plot(modelx, modely_med, 'b-', alpha=0.7, label=r'$\int S\mathrm{dv}_\mathrm{med}$ = %.3f Jy km s$^{-1}$' % (Sdv_model_med))
    
    # Mean        
    modely_mean = Bmodelx(modelx, av_best_par[0][0], av_best_par[1][0], av_best_par[2][0], av_best_par[3][0], 
                            av_best_par[4][0], av_best_par[5][0], av_best_par[6][0], med_best_par[7][0])
    # Integrated flux of mean model
    Sdv_model_mean = np.sum(modely_mean*dvmodel)*all_max_flux[d]
    pl.plot(modelx, modely_mean, 'c--', alpha=0.7, label=r'$\int S\mathrm{dv}_\mathrm{mean}$ = %.3f Jy km s$^{-1}$' % (Sdv_model_mean)) 
            
    # Non sampled fit with closest matching Sdv   
    modely_sdv = Bmodelx(modelx, sdv_models[d][0], sdv_models[d][1], sdv_models[d][2], sdv_models[d][3], 
                            sdv_models[d][4], sdv_models[d][5], sdv_models[d][6], sdv_models[d][7])
    pl.plot(modelx, modely_sdv, 'r-', alpha=0.7, label=r'$\int S\mathrm{dv}_\mathrm{non-sampled \ fit}$ = %.3f Jy km s$^{-1}$' % (sdv_models[d][8]))  
    
    # Sampled fit with closest matching Sdv
    modely_sampled_sdv = Bmodelx(modelx, sdv_sampled_models[d][0], sdv_sampled_models[d][1], sdv_sampled_models[d][2], sdv_sampled_models[d][3], 
                            sdv_sampled_models[d][4], sdv_sampled_models[d][5], sdv_sampled_models[d][6], sdv_sampled_models[d][7])
    pl.plot(modelx, modely_sampled_sdv, 'g-', alpha=0.7, label=r'$\int S\mathrm{dv}_\mathrm{sampled \ fit}$ = %.3f Jy km s$^{-1}$' % (sdv_sampled_models[d][8]))  
            
    pl.xlabel(r'Velocity [km s$^{-1}$]', fontsize=28)
    pl.ylabel('Normalized Flux Density', fontsize=28)
    pl.gca().tick_params(axis='both', labelsize=28)
    plt.tight_layout()
    
    # Axes parameters
    pl.gca().xaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().yaxis.set_minor_locator(AutoMinorLocator())
    pl.gca().tick_params(axis='both', which='major', direction='in', length=10, top=True, right=True)
    pl.gca().tick_params(axis='both', which='minor', direction='in', length=5, top=True, right=True)
    pl.legend(loc='upper left', fontsize=15, frameon=False)
    
    # Anchored text
#     textstr = '\n'.join((
#     r'Av. W$_{50}$ = %.2f $\pm$ %.2f km s$^{-1}$' % (2*av_best_par[3][0], 2*av_best_par[3][1], ),
#     r'$\int S\mathrm{dv}_\mathrm{data}$ = %.3f $\pm$ %.3f Jy km s$^{-1}$' % (Sdv_data, Sdv_data_err, ),
#     r'$\int S\mathrm{dv}_\mathrm{model}$ = %.3f Jy km s$^{-1}$' % (Sdv_model, )))
#     pl.annotate(textstr, xy=(0.03, 0.7), xycoords='axes fraction', fontsize=17)
    
    pl.tight_layout()
    
    pl.savefig(dest+'med_b1b2xexpn_best_par'+str(detections[d]['Detection'])+'.png')
    pl.close()